<a href="https://colab.research.google.com/github/arthlz/Burnout-Risk-Machine-Learning/blob/main/Front-end%2BBack_end/Back_End.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Baixando modelo direto do repositório no GitHub

In [ ]:
import os

url_raw = "https://raw.githubusercontent.com/arthlz/Burnout-Risk-Machine-Learning/main/modelo_burnout_final.pkl"

if not os.path.exists('modelo_burnout_final.pkl'):
    os.system(f"wget {url_raw}")
    print("✅ Download concluído!")
else:
    print("Arquivo já existe no diretório.")

✅ Download concluído!


Instalação de bibliotecas essenciais para a conexão com o Front-End

In [ ]:
!pip install flask-cors pyngrok

  Using cached flask_cors-6.0.2-py3-none-any.whl.metadata (5.3 kB)
  Using cached pyngrok-7.5.0-py3-none-any.whl.metadata (8.1 kB)
Using cached flask_cors-6.0.2-py3-none-any.whl (13 kB)
Using cached pyngrok-7.5.0-py3-none-any.whl (24 kB)

   ---------------------------------------- 0/7 [Werkzeug]
   ---------------------------------------- 0/7 [Werkzeug]
   ---------------------------------------- 0/7 [Werkzeug]
   ---------------------------------------- 0/7 [Werkzeug]
   ----- ---------------------------------- 1/7 [pyngrok]
   ----------- ---------------------------- 2/7 [itsdangerous]
   ----------------- ---------------------- 3/7 [click]
   ---------------------- ----------------- 4/7 [blinker]
   ---------------------------- ----------- 5/7 [flask]
   ---------------------------- ----------- 5/7 [flask]
   ---------------------------------- ----- 6/7 [flask-cors]
   ---------------------------------------- 7/7 [flask-cors]



"Limpar" conexões extras

In [ ]:
!pkill -9 ngrok

Código do Back-End

In [ ]:
import joblib
import os
from flask import Flask, request, jsonify
from flask_cors import CORS
from pyngrok import ngrok


caminho_arquivo = 'modelo_burnout_final.pkl'
# 1. Carregar o modelo
try:
    modelo_xgb = joblib.load(caminho_arquivo)
    print("✅ Modelo carregado com sucesso!")
except:
    print("❌ ERRO: Arquivo 'modelo_burnout.pkl' não encontrado na aba de arquivos.")

app = Flask(__name__)
CORS(app)

@app.route('/predict', methods=['POST'])
def predict():
    try:

        dados = request.json

        # 2. Tradução dos dados (Label Encoding)
        genero = 1 if dados['genero'] == 'm' else 0
        empresa = 1 if dados['tipo_empresa'] == 's' else 0
        home = 1 if dados['home_office'] == 's' else 0

        # 3. Organização das features
        features = [[
            genero,                         # Gender
            empresa,                        # Company Type
            home,                           # WFH Setup
            float(dados['nivel_cargo']),    # Designation
            float(dados['horas_trabalho']), # Resource Allocation
            float(dados['fadiga_mental']),   # Mental Fatigue Score
            float(empresa*dados['nivel_cargo']/10)
        ]]

        # 4. PREDIÇÃO
        previsao = modelo_xgb.predict(features)[0]
        previsao = 1 if previsao > 1 else previsao
        # Garante que o resultado seja um número comum (float)

        return jsonify({
            'burnout_level': float(previsao),
            'status': 'sucesso'
        })

    except Exception as e:
        return jsonify({'error': str(e), 'status': 'erro'}), 400

# Inicialização do servidor

# 5. Autenticação Ngrok
NGROK_AUTH_TOKEN = "38oGVTLulfbspCl6Dln9qlX1PtM_5DdSdukFWqKaE7okf2q69"
ngrok.set_auth_token(NGROK_AUTH_TOKEN)

# 6. Rodar Servidor
try:
    # Fecha conexões antigas
    ngrok.kill()
    public_url = ngrok.connect(5000)
    print(f"\n🚀 BACKEND ONLINE!")
    print(f"🔗 URL PARA O SEU JS: {public_url.public_url}/predict")
    print(f"--------------------------------------------------")

    app.run()
except Exception as e:
    print(f"Erro ao iniciar o Ngrok: {e}")

c:\Users\Desenvolvedor\anaconda3\Lib\pickle.py:1760: UserWarning: [18:23:14] WARNING: C:\b\abs_d97hy_84m6\croot\xgboost-split_1749630932152\work\src\gbm\../common/error_msg.h:82: If you are loading a serialized model (like pickle in Python, RDS in R) or
configuration generated by an older version of XGBoost, please export the model by calling
`Booster.save_model` from that version first, then load it back in current version. See:

    https://xgboost.readthedocs.io/en/stable/tutorials/saving_model.html

for more details about differences between saving model and serializing.

  setstate(state)


✅ Modelo carregado com sucesso!

🚀 BACKEND ONLINE!
🔗 URL PARA O SEU JS: https://maryalice-anthropometric-rufina.ngrok-free.dev/predict
--------------------------------------------------
 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [30/Jan/2026 18:23:23] "GET / HTTP/1.1" 404 -
